In [57]:
# ---------------------------------------------------------
# Import declaration.
# ---------------------------------------------------------

import pandas as pd
import numpy as np
from IPython.display import Image


In [58]:
# ----------------------------------------------------------
# Load the data from jester-data-1.csv.
# ----------------------------------------------------------

jokes_number = np.linspace(0, 100, 101)
#jokes_number = np.append(jokes_number, "test_set")
print (jokes_number)
j_df = pd.read_csv("jester-data-1.csv")
j_df.columns = jokes_number
j_df.head()

# ----------------------------------------------------------
# Split data into test_set and training_set based on giving label 99
# Firstly, Copy j_df as j_test_df and j_train_df
# Scondly, Split and label datas simply using 2*for loops and np.random.rand()
# ----------------------------------------------------------

j_shape = j_df.shape
train_values = j_df.values # Copy data values from j_df as train_values prepared for next step
test_values = j_df.values # Copy data values from j_df as test_values prepared for next step

print(j_shape[0])
print(train_values)


# Label data 99 in train_set and keep that data labeling others 99 in test_set

for i in range (0, j_shape[0]):
    for j in range (0, j_shape[1]):
        if np.random.rand() > 0.9:
            train_values[i,j] = 99
        else:
            test_values[i, j] = 99

print(train_values)
print(test_values)

j_train_df = pd.DataFrame(train_values)
j_test_df = pd.DataFrame(test_values)

print(j_train_df)
print(j_test_df)

# End of splitting data

[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.
  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.
  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.
  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.
  98.  99. 100.]
24982
[[100.     4.08  -0.29 ...   0.34  -4.32   1.07]
 [ 49.    99.    99.   ...  99.    99.    99.  ]
 [ 48.    99.     8.35 ...  99.    99.    99.  ]
 ...
 [ 39.    99.    99.   ...  99.    99.    99.  ]
 [ 37.    99.    99.   ...  99.    99.    99.  ]
 [ 72.     2.43   2.67 ...  99.    99.    99.  ]]
[[100.     4.08  -0.29 ...  99.    -4.32   1.07]
 [ 49.    99.    99.   ...  99.    99.    99.  ]
 [ 48.    99.     8.35 ...  99.    99.    99.  ]
 ...
 [ 39.    99.    99.   ...  

In [59]:
# -------------------------------------------------------------
# Create a model to infer the hidden rating of users.
# (As it would take a long time for system to train, a better modeling method shoud be arose.)
# Firstly, let's assume how many hidden features do a joke have, and how many hidden standards do a costumer have(2).
# -------------------------------------------------------------

# Let's say the numbers of features of joke & costumer is 2
n_features = 2 

# Then we create 2 lists ofrandom values for both hidden features & standards
reader_ratings = train_values
latent_reader_preferences = np.random.random((j_shape[0], n_features))
latent_jester_features = np.random.random((j_shape[1],n_features))

# Print them so we'll see 24982 readers have 24982*2 hidden standards.
# 101 Jesters have 101*2 hidden features.
print(latent_readers_preferences.shape)
print(latent_jester_features.shape)

(24982, 2)
(101, 2)


In [60]:
# -------------------------------------------------------------
# Secondly, create the training fuctions.
# Functions should at least include 2.
# The first one should generate a rating given reader no.and jester no..
# The second one would use all the datas we have to train and give a best results.
# -------------------------------------------------------------

# Predict a rating given a reader_id and a jester_id(with dot).
def predict_rating_dot(reader_id, jester_id):
    reader_preference = latent_reader_preferences[reader_id]
    jester_features = latent_jester_features[jester_id]
    return reader_preference.dot(jester_features)

# Predict a rating given a reader_id and a jester_id(with functions).
def predict_rating_function(reader_id, jester_id):
    reader_preference = latent_reader_preferences[reader_id]
    jester_features = latent_jester_features[jester_id]
    rating = reader_preference[0]*jester_features[0] + reader_preference[1]*jester_features[1]
    return rating

# Caution: two rating generators are absolutely the same!

# Training function.
def train(reader_id, jester_id, rating, alpha = 0.0001):
    
    prediction_rating = predict_rating_dot(reader_id, jester_id)
    err =  ( prediction_rating- rating ); # give a error
    
    reader_pref_values = latent_reader_preferences[reader_id][:] # pref value record.
    
    # Here we update the hidden features & standards.
    latent_reader_preferences[reader_id] -= alpha * err * latent_jester_features[jester_id]
    latent_jester_features[jester_id] -= alpha * err * reader_pref_values
    
    return err

print(predict_rating_dot(0, 0), predict_rating_function(0, 0))

train(0, 0, reader_ratings[0, 0])

print(predict_rating_dot(0, 0), predict_rating_function(0, 0))

0.9917083972337077 0.9917083972337077
1.0124998375095864 1.0124998375095864


In [61]:
# ----------------------------------------------------------------
# Finally, we train all our data in a loop.
# ----------------------------------------------------------------

# In the loop, we should not use rating value = 99
# Plus, we should predict it also.
def sgd(iterations = 500):
    for iteration in range(0,iterations):
        error = []
        for reader_id in range(0,latent_reader_preferences.shape[0]):
            for jester_id in range(0,latent_jester_features.shape[0]):
                rating = reader_ratings[reader_id][jester_id]
                if (rating != 99):
                    err = train(reader_id, jester_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        print(mse)


In [62]:
# --------------------------------------------------------------------
# Now let's see what would we get.
# Wish we could get our results very fast.(23333333333)
# --------------------------------------------------------------------
sgd()

59.774833350676815
31.836082196875214
25.441344540165602
24.706722756411068
24.628452955159307
24.609146560084508
24.596147859124155
24.584334771182522
24.57303779543364
24.562050321830494
24.551188469462275
24.540207313775745
24.528748932174956
24.51627519592497
24.501967132415956
24.484571983793405
24.462172357608427
24.431843121168683
24.38915470635413
24.32748573573884
24.23714624468897
24.10443301443224
23.91102064145771
23.634614054516064
23.252452953631174
22.74937473990906
22.130088108469458
21.42987185950759
20.712727950378945
20.050918806655616
19.49686815743463
19.06824170195644
18.753891845189475
18.529659110829552
18.370914849941116
18.257949268204705
18.17661253591182
18.117199152973143
18.073138054922364
18.039967994652553
18.01463112795958
17.99500614927974
17.979602446279713
17.96735886472494
17.957510127637516
17.949497265878705
17.942906930971333
17.93742983530864
17.93283197480438
17.92893447280653
17.92559929503513
17.92271900374521
17.92020932108865
17.91800366842

KeyboardInterrupt: 